In [1]:
from array import array
import subprocess
import math
import ROOT
import os
import numpy as np

Welcome to JupyROOT 6.29/01


In [2]:
!pwd

/home/mdessole/Projects/ROOT/genvectorxfork/plots


In [3]:
buildcuda = '/home/mdessole/Projects/ROOT/genvectorxfork/build_cuda/'

In [4]:
buildoneapi = '/home/mdessole/Projects/ROOT/genvectorxfork/build_oneapi/'

In [5]:
buildopensycl = '/home/mdessole/Projects/ROOT/genvectorxfork/build_opensycl/'

In [ ]:
# must be set before starting the kernel
os.environ['ACPP_VISIBILITY_MASK']='cuda'

In [6]:
!ls /home/mdessole/Projects/ROOT/genvectorx/build_opensycl/testx

CMakeFiles	 InvariantMassesSYCL  SInvariantMassesSYCL
InvariantMasses  SInvariantMasses     cmake_install.cmake


In [7]:
!ls /home/mdessole/Projects/ROOT/genvectorx/build_oneapi/testx

CMakeFiles	 InvariantMassesSYCL  SInvariantMassesSYCL
InvariantMasses  SInvariantMasses     cmake_install.cmake


In [8]:
!echo $LD_LIBRARY_PATH

/opt/intel/oneapi/tbb/2021.10.0/env/../lib/intel64/gcc4.8:/opt/intel/oneapi/debugger/2023.2.0/gdb/intel64/lib:/opt/intel/oneapi/debugger/2023.2.0/libipt/intel64/lib:/opt/intel/oneapi/debugger/2023.2.0/dep/lib:/opt/intel/oneapi/compiler/2023.2.1/linux/lib:/opt/intel/oneapi/compiler/2023.2.1/linux/lib/x64:/opt/intel/oneapi/compiler/2023.2.1/linux/lib/oclfpga/host/linux64/lib:/opt/intel/oneapi/compiler/2023.2.1/linux/compiler/lib/intel64_lin:/home/mdessole/Projects/AdaptiveCpp/AdaptiveCpp_install/lib:/usr/local/cuda-11.8/lib64:/home/mdessole/Projects/ROOT/root/lib:/opt/rocm-5.7.0/lib:/opt/rocm-5.7.0/opencl/lib


In [9]:
sizes = array('d',[1000, 10000, 100000,1000000,5000000,10000000, 50000000,100000000]) #10,100,1000,10000,100000,
n = len(sizes)

In [10]:
Ssizes = array('d',[1000, 10000, 100000,1000000,5000000,10000000, 50000000,100000000,200000000]) #10,100,1000,10000,100000,
n = len(sizes)

In [11]:
def result2list(exe, N, nruns):
    print(exe, str(int(N)), str(int(nruns)))
    result = subprocess.run([exe, str(int(N)), str(int(nruns)) ], stdout=subprocess.PIPE).stdout.decode('utf-8')
    print(result)
    if len(result)<=0:
        return []
    result = result.replace("\r","").replace("\n","")
    time = np.zeros(nruns)
    for r in range(nruns):
        i = result.find("time")
        j = result.find("(s)")
        if (i<0) or (j<0):
            break
        time[r] = float(result[i+5:j])
        result = result[j+3:]
    #endif
    return time

In [12]:
def run_benchmark(builddir, sizes, nruns = 1, testname = "InvariantMasses", btype = 'cpu'):
    timeopt = array( 'd' )
    timestd = array( 'd' )
    
    if btype == 'cpu':
        exe = os.path.join(builddir,'testx',testname)
    elif btype == 'cuda':
        exe = os.path.join(builddir,'testx',testname+'CUDA')
    elif btype == 'sycl':
        exe = os.path.join(builddir,'testx',testname+'SYCL')
    #endif
    
    if (not os.path.exists(exe)):
        print("Executable", exe, "not found!")
        return timeopt, timestd
    
    for i, N in enumerate(sizes):
        time = result2list(exe,N,nruns)
        if (len(time)<=0):
            return timeopt, timestd
        print(time, '\n')
        timeopt.append(np.amin(time))
        timestd.append(np.std(time)) 
    #endfor
    return timeopt, timestd

In [19]:
timecpu, stdcpu = run_benchmark(buildoneapi, sizes, nruns = 3, btype = 'cpu')

/home/mdessole/Projects/ROOT/genvectorxfork/build_oneapi/testx/InvariantMasses 1000 3
RVec cpu time 0.000115 (s)
RVec cpu time 0.000111 (s)
RVec cpu time 6.1e-05 (s)

[1.15e-04 1.11e-04 6.10e-05] 

/home/mdessole/Projects/ROOT/genvectorxfork/build_oneapi/testx/InvariantMasses 10000 3
RVec cpu time 0.000592 (s)
RVec cpu time 0.000591 (s)
RVec cpu time 0.000591 (s)

[0.000592 0.000591 0.000591] 

/home/mdessole/Projects/ROOT/genvectorxfork/build_oneapi/testx/InvariantMasses 100000 3
RVec cpu time 0.005935 (s)
RVec cpu time 0.005932 (s)
RVec cpu time 0.006001 (s)

[0.005935 0.005932 0.006001] 

/home/mdessole/Projects/ROOT/genvectorxfork/build_oneapi/testx/InvariantMasses 1000000 3
RVec cpu time 0.061837 (s)
RVec cpu time 0.059077 (s)
RVec cpu time 0.058994 (s)

[0.061837 0.059077 0.058994] 

/home/mdessole/Projects/ROOT/genvectorxfork/build_oneapi/testx/InvariantMasses 5000000 3
RVec cpu time 0.313937 (s)
RVec cpu time 0.314618 (s)
RVec cpu time 0.314248 (s)

[0.313937 0.314618 0.314248]

In [20]:
timecpu

array('d', [6.1e-05, 0.000591, 0.005932, 0.058994, 0.313937, 0.628339, 3.13605, 6.27893])

In [21]:
Stimecpu, Sstdcpu = run_benchmark(buildoneapi, Ssizes, testname = "SInvariantMasses", nruns = 3, btype = 'cpu')

/home/mdessole/Projects/ROOT/genvectorxfork/build_oneapi/testx/SInvariantMasses 1000 3
SINGLE_PRECISION defined 
RVec cpu time 4.4e-05 (s)
RVec cpu time 4.5e-05 (s)
RVec cpu time 4.4e-05 (s)

[4.4e-05 4.5e-05 4.4e-05] 

/home/mdessole/Projects/ROOT/genvectorxfork/build_oneapi/testx/SInvariantMasses 10000 3
SINGLE_PRECISION defined 
RVec cpu time 0.000436 (s)
RVec cpu time 0.000436 (s)
RVec cpu time 0.000454 (s)

[0.000436 0.000436 0.000454] 

/home/mdessole/Projects/ROOT/genvectorxfork/build_oneapi/testx/SInvariantMasses 100000 3
SINGLE_PRECISION defined 
RVec cpu time 0.004354 (s)
RVec cpu time 0.004381 (s)
RVec cpu time 0.004348 (s)

[0.004354 0.004381 0.004348] 

/home/mdessole/Projects/ROOT/genvectorxfork/build_oneapi/testx/SInvariantMasses 1000000 3
SINGLE_PRECISION defined 
RVec cpu time 0.04472 (s)
RVec cpu time 0.043394 (s)
RVec cpu time 0.043474 (s)

[0.04472  0.043394 0.043474] 

/home/mdessole/Projects/ROOT/genvectorxfork/build_oneapi/testx/SInvariantMasses 5000000 3
SINGLE_

In [22]:
Stimecpu

array('d', [4.4e-05, 0.000436, 0.004348, 0.043394, 0.225247, 0.455478, 2.27663, 4.56178, 9.09319])

In [23]:
timeoneapi, stdoneapi = run_benchmark(buildoneapi, sizes, nruns = 3, btype = 'sycl')

/home/mdessole/Projects/ROOT/genvectorxfork/build_oneapi/testx/InvariantMassesSYCL 1000 3
ROOT_MEAS_TIMING defined 
ROOT_MATH_SYCL defined 
sycl::queue check - selected device:
NVIDIA GeForce RTX 3060
sycl time 0.000364 (s)
sycl time 5.2e-05 (s)
sycl time 0.000135 (s)

[3.64e-04 5.20e-05 1.35e-04] 

/home/mdessole/Projects/ROOT/genvectorxfork/build_oneapi/testx/InvariantMassesSYCL 10000 3
ROOT_MEAS_TIMING defined 
ROOT_MATH_SYCL defined 
sycl::queue check - selected device:
NVIDIA GeForce RTX 3060
sycl time 0.000453 (s)
sycl time 0.000106 (s)
sycl time 0.000189 (s)

[0.000453 0.000106 0.000189] 

/home/mdessole/Projects/ROOT/genvectorxfork/build_oneapi/testx/InvariantMassesSYCL 100000 3
ROOT_MEAS_TIMING defined 
ROOT_MATH_SYCL defined 
sycl::queue check - selected device:
NVIDIA GeForce RTX 3060
sycl time 0.001401 (s)
sycl time 0.001048 (s)
sycl time 0.000956 (s)

[0.001401 0.001048 0.000956] 

/home/mdessole/Projects/ROOT/genvectorxfork/build_oneapi/testx/InvariantMassesSYCL 1000000 3

In [24]:
timeoneapi

array('d', [5.2e-05, 0.000106, 0.000956, 0.00838, 0.042846, 0.085533, 0.414523, 0.817115])

In [25]:
Stimeoneapi, Sstdoneapi = run_benchmark(buildoneapi, Ssizes, testname = "SInvariantMasses", nruns = 3, btype = 'sycl')

/home/mdessole/Projects/ROOT/genvectorxfork/build_oneapi/testx/SInvariantMassesSYCL 1000 3
ROOT_MEAS_TIMING defined 
ROOT_MATH_SYCL defined 
SINGLE_PRECISION defined 
sycl::queue check - selected device:
NVIDIA GeForce RTX 3060
sycl time 0.000224 (s)
sycl time 4.3e-05 (s)
sycl time 0.000124 (s)

[2.24e-04 4.30e-05 1.24e-04] 

/home/mdessole/Projects/ROOT/genvectorxfork/build_oneapi/testx/SInvariantMassesSYCL 10000 3
ROOT_MEAS_TIMING defined 
ROOT_MATH_SYCL defined 
SINGLE_PRECISION defined 
sycl::queue check - selected device:
NVIDIA GeForce RTX 3060
sycl time 0.000267 (s)
sycl time 7.7e-05 (s)
sycl time 0.000153 (s)

[2.67e-04 7.70e-05 1.53e-04] 

/home/mdessole/Projects/ROOT/genvectorxfork/build_oneapi/testx/SInvariantMassesSYCL 100000 3
ROOT_MEAS_TIMING defined 
ROOT_MATH_SYCL defined 
SINGLE_PRECISION defined 
sycl::queue check - selected device:
NVIDIA GeForce RTX 3060
sycl time 0.000745 (s)
sycl time 0.000488 (s)
sycl time 0.000485 (s)

[0.000745 0.000488 0.000485] 

/home/mdesso

In [13]:
timeopensycl, stdopensycl = run_benchmark(buildopensycl, sizes,  nruns = 3, btype = 'sycl')

/home/mdessole/Projects/ROOT/genvectorxfork/build_opensycl/testx/InvariantMassesSYCL 1000 3
ROOT_MEAS_TIMING defined 
ROOT_MATH_SYCL defined 
sycl::queue check - selected device:
NVIDIA GeForce RTX 3060
sycl time 0.00098 (s)
sycl time 0.000109 (s)
sycl time 0.0002 (s)

[0.00098  0.000109 0.0002  ] 

/home/mdessole/Projects/ROOT/genvectorxfork/build_opensycl/testx/InvariantMassesSYCL 10000 3
ROOT_MEAS_TIMING defined 
ROOT_MATH_SYCL defined 
sycl::queue check - selected device:
NVIDIA GeForce RTX 3060
sycl time 0.001027 (s)
sycl time 0.00018 (s)
sycl time 0.000188 (s)

[0.001027 0.00018  0.000188] 

/home/mdessole/Projects/ROOT/genvectorxfork/build_opensycl/testx/InvariantMassesSYCL 100000 3
ROOT_MEAS_TIMING defined 
ROOT_MATH_SYCL defined 
sycl::queue check - selected device:
NVIDIA GeForce RTX 3060
sycl time 0.00211 (s)
sycl time 0.00124 (s)
sycl time 0.001121 (s)

[0.00211  0.00124  0.001121] 

/home/mdessole/Projects/ROOT/genvectorxfork/build_opensycl/testx/InvariantMassesSYCL 100000

In [14]:
timeopensycl

array('d', [0.000109, 0.00018, 0.001121, 0.011112, 0.050796, 0.095799, 0.46526, 0.928883])

In [15]:
Stimeopensycl, Sstdopensycl = run_benchmark(buildopensycl, Ssizes, testname = "SInvariantMasses", nruns = 3, btype = 'sycl')

/home/mdessole/Projects/ROOT/genvectorxfork/build_opensycl/testx/SInvariantMassesSYCL 1000 3
ROOT_MEAS_TIMING defined 
ROOT_MATH_SYCL defined 
SINGLE_PRECISION defined 
sycl::queue check - selected device:
NVIDIA GeForce RTX 3060
sycl time 0.087188 (s)
sycl time 0.000173 (s)
sycl time 0.00015 (s)

[0.087188 0.000173 0.00015 ] 

/home/mdessole/Projects/ROOT/genvectorxfork/build_opensycl/testx/SInvariantMassesSYCL 10000 3
ROOT_MEAS_TIMING defined 
ROOT_MATH_SYCL defined 
SINGLE_PRECISION defined 
sycl::queue check - selected device:
NVIDIA GeForce RTX 3060
sycl time 0.000898 (s)
sycl time 0.00014 (s)
sycl time 0.000173 (s)

[0.000898 0.00014  0.000173] 

/home/mdessole/Projects/ROOT/genvectorxfork/build_opensycl/testx/SInvariantMassesSYCL 100000 3
ROOT_MEAS_TIMING defined 
ROOT_MATH_SYCL defined 
SINGLE_PRECISION defined 
sycl::queue check - selected device:
NVIDIA GeForce RTX 3060
sycl time 0.001402 (s)
sycl time 0.000574 (s)
sycl time 0.000568 (s)

[0.001402 0.000574 0.000568] 

/home/

In [16]:
timecuda, stdcuda = run_benchmark(buildcuda, sizes, nruns = 3, btype = 'cuda')

/home/mdessole/Projects/ROOT/genvectorxfork/build_cuda/testx/InvariantMassesCUDA 1000 3
ROOT_MEAS_TIMING defined 
ROOT_MATH_CUDA defined 
CUDA available devices: 1
cuda time 0.000271 (s)
cuda time 4.4e-05 (s)
cuda time 4.2e-05 (s)

[2.71e-04 4.40e-05 4.20e-05] 

/home/mdessole/Projects/ROOT/genvectorxfork/build_cuda/testx/InvariantMassesCUDA 10000 3
ROOT_MEAS_TIMING defined 
ROOT_MATH_CUDA defined 
CUDA available devices: 1
cuda time 0.00041 (s)
cuda time 0.000185 (s)
cuda time 0.00017 (s)

[0.00041  0.000185 0.00017 ] 

/home/mdessole/Projects/ROOT/genvectorxfork/build_cuda/testx/InvariantMassesCUDA 100000 3
ROOT_MEAS_TIMING defined 
ROOT_MATH_CUDA defined 
CUDA available devices: 1
cuda time 0.001659 (s)
cuda time 0.00139 (s)
cuda time 0.001273 (s)

[0.001659 0.00139  0.001273] 

/home/mdessole/Projects/ROOT/genvectorxfork/build_cuda/testx/InvariantMassesCUDA 1000000 3
ROOT_MEAS_TIMING defined 
ROOT_MATH_CUDA defined 
CUDA available devices: 1
cuda time 0.011725 (s)
cuda time 0.01104

In [17]:
timecuda

array('d', [4.2e-05, 0.00017, 0.001273, 0.011048, 0.055299, 0.106399, 0.527184, 1.06491])

In [18]:
Stimecuda, Sstdcuda = run_benchmark(buildcuda, Ssizes, nruns = 3, testname = "SInvariantMasses", btype = 'cuda')

/home/mdessole/Projects/ROOT/genvectorxfork/build_cuda/testx/SInvariantMassesCUDA 1000 3
ROOT_MEAS_TIMING defined 
ROOT_MATH_CUDA defined 
SINGLE_PRECISION defined 
CUDA available devices: 1
cuda time 0.000229 (s)
cuda time 2.9e-05 (s)
cuda time 2.7e-05 (s)

[2.29e-04 2.90e-05 2.70e-05] 

/home/mdessole/Projects/ROOT/genvectorxfork/build_cuda/testx/SInvariantMassesCUDA 10000 3
ROOT_MEAS_TIMING defined 
ROOT_MATH_CUDA defined 
SINGLE_PRECISION defined 
CUDA available devices: 1
cuda time 0.000286 (s)
cuda time 9.4e-05 (s)
cuda time 8.1e-05 (s)

[2.86e-04 9.40e-05 8.10e-05] 

/home/mdessole/Projects/ROOT/genvectorxfork/build_cuda/testx/SInvariantMassesCUDA 100000 3
ROOT_MEAS_TIMING defined 
ROOT_MATH_CUDA defined 
SINGLE_PRECISION defined 
CUDA available devices: 1
cuda time 0.000816 (s)
cuda time 0.000608 (s)
cuda time 0.000575 (s)

[0.000816 0.000608 0.000575] 

/home/mdessole/Projects/ROOT/genvectorxfork/build_cuda/testx/SInvariantMassesCUDA 1000000 3
ROOT_MEAS_TIMING defined 
ROOT_MA

In [26]:
n = len(sizes) 
speedupopensycl = array('d', [timecpu[i]/timeopensycl[i] for i in range(n)])
speeduponeapi   = array('d', [timecpu[i]/timeoneapi[i] for i in range(n)])
speedupcuda     = array('d', [timecpu[i]/timecuda[i] for i in range(n)])

In [27]:
n = len(Ssizes)
Sspeedupopensycl = array('d', [Stimecpu[i]/Stimeopensycl[i] for i in range(n)])
Sspeeduponeapi   = array('d', [Stimecpu[i]/Stimeoneapi[i] for i in range(n)])
Sspeedupcuda     = array('d', [Stimecpu[i]/Stimecuda[i] for i in range(n)])

In [28]:
print(Sspeedupopensycl)
print(Sspeeduponeapi)
print(Sspeedupcuda)

array('d', [0.29333333333333333, 3.1142857142857148, 7.654929577464789, 7.599649737302978, 10.691935254189017, 10.533718778908417, 10.886039027795707, 10.897394245745463, 10.921058478626135])
array('d', [1.0232558139534884, 5.662337662337663, 8.964948453608248, 9.943629697525207, 10.91683225900257, 10.83516901777006, 11.113210126038524, 11.029342075995395, 11.142536445613722])
array('d', [1.6296296296296295, 5.382716049382716, 7.561739130434783, 9.596196373286157, 10.630875967528791, 10.643501425433472, 10.678777815302638, 10.649307598210866, 10.643874837295977])


In [29]:
val = list(sizes)
sizes_gb = [v*8*4/1e9 for v in val] #(nb of doubles)*(bytes for doubles)*(4=dim of LVector)/(bytes in GB)
sizes_gb = array('d', sizes_gb)
sizes_gb

array('d', [3.2e-05, 0.00032, 0.0032, 0.032, 0.16, 0.32, 1.6, 3.2])

In [30]:
val = list(Ssizes)
Ssizes_gb = [v*4*4/1e9 for v in val] #(nb of floats)*(bytes for floats)*(4=dim of LVector)/(bytes in GB)
Ssizes_gb = array('d', Ssizes_gb)
Ssizes_gb

array('d', [1.6e-05, 0.00016, 0.0016, 0.016, 0.08, 0.16, 0.8, 1.6, 3.2])

In [31]:
p = ROOT.TGraph(len(sizes), sizes, speedupopensycl)
g = ROOT.TGraph(len(sizes), sizes, speeduponeapi)
q = ROOT.TGraph(len(sizes), sizes, speedupcuda)


c4 = ROOT.TCanvas()
g.SetMarkerStyle(7)
g.SetLineColor(ROOT.kBlue)
p.SetMarkerStyle(5)
p.SetLineColor(ROOT.kRed)
q.SetMarkerStyle(3)
q.SetLineColor(ROOT.kGreen)


q.SetTitle("Speedup wrt RVec")
q.GetXaxis().SetTitle( 'Input size (# doubles)' )
q.GetYaxis().SetTitle( 'Speedup' )

p.GetYaxis().SetRangeUser(0,6)

leg = ROOT.TLegend(0.7, 0.75, 0.30, 0.87)
leg.AddEntry(p,"OpenSYCL","L")
leg.AddEntry(g,"OneAPI","L")
leg.AddEntry(q,"CUDA","L")


c4.SetLogx();

q.Draw()
g.Draw('LP')
p.Draw('LP')
leg.Draw()
c4.Draw()

c4.SaveAs("speedup.png")

Info in <TCanvas::Print>: png file speedup.png has been created


In [53]:
len(Ssizes), Ssizes, len(Sspeedupopensycl)

(9,
 array('d', [1000.0, 10000.0, 100000.0, 1000000.0, 5000000.0, 10000000.0, 50000000.0, 100000000.0, 200000000.0]),
 8)

In [32]:
p = ROOT.TGraph(len(Ssizes), Ssizes, Sspeedupopensycl)
g = ROOT.TGraph(len(Ssizes), Ssizes, Sspeeduponeapi)
q = ROOT.TGraph(len(Ssizes), Ssizes, Sspeedupcuda)


c4 = ROOT.TCanvas()
g.SetMarkerStyle(7)
g.SetLineColor(ROOT.kBlue)
p.SetMarkerStyle(5)
p.SetLineColor(ROOT.kRed)
q.SetMarkerStyle(3)
q.SetLineColor(ROOT.kGreen)


q.SetTitle("Speedup wrt RVec - Single Precision")
q.GetXaxis().SetTitle( 'Input size (# floats)' )
q.GetYaxis().SetTitle( 'Speedup' )

p.GetYaxis().SetRangeUser(0,6)

leg = ROOT.TLegend(0.7, 0.75, 0.30, 0.87)
leg.AddEntry(p,"OpenSYCL","L")
leg.AddEntry(g,"OneAPI","L")
leg.AddEntry(q,"CUDA","L")


c4.SetLogx();

q.Draw()
g.Draw('LP')
p.Draw('LP')
leg.Draw()
c4.Draw()

c4.SaveAs("Sspeedup.png")

Info in <TCanvas::Print>: png file Sspeedup.png has been created


In [33]:
p = ROOT.TGraphErrors(len(sizes), sizes, timeopensycl, stdopensycl)
g = ROOT.TGraphErrors(len(sizes), sizes, timeoneapi, stdoneapi)
q = ROOT.TGraphErrors(len(sizes), sizes, timecuda, stdcuda)
r = ROOT.TGraphErrors(len(sizes), sizes, timecpu, stdcpu)


c4 = ROOT.TCanvas()
g.SetLineColor(ROOT.kBlue)
p.SetLineColor(ROOT.kRed)
q.SetLineColor(ROOT.kGreen)


r.SetTitle("Execution times")
r.GetXaxis().SetTitle( 'Input size (# doubles)' )
r.GetYaxis().SetTitle( 'Time (s)' )

p.GetYaxis().SetRangeUser(0,6)

leg = ROOT.TLegend(0.7, 0.75, 0.30, 0.87)
leg.AddEntry(r,"RVec","L")
leg.AddEntry(p,"OpenSYCL","L")
leg.AddEntry(g,"OneAPI","L")
leg.AddEntry(q,"CUDA","L")

c4.SetLogy();
c4.SetLogx();

r.Draw()
q.Draw('LP')
g.Draw('LP')
p.Draw('LP')
leg.Draw()
c4.Draw()

c4.SaveAs("exectimes.png")

Info in <TCanvas::Print>: png file exectimes.png has been created


In [34]:
p = ROOT.TGraphErrors(len(sizes), sizes, Stimeopensycl, Sstdopensycl)
g = ROOT.TGraphErrors(len(sizes), sizes, Stimeoneapi, Sstdoneapi)
q = ROOT.TGraphErrors(len(sizes), sizes, Stimecuda, Sstdcuda)
r = ROOT.TGraphErrors(len(sizes), sizes, Stimecpu, Sstdcpu)


c4 = ROOT.TCanvas()
g.SetLineColor(ROOT.kBlue)
p.SetLineColor(ROOT.kRed)
q.SetLineColor(ROOT.kGreen)


r.SetTitle("Execution times - Single Precision")
r.GetXaxis().SetTitle( 'Input size (# floats)' )
r.GetYaxis().SetTitle( 'Time (s)' )

p.GetYaxis().SetRangeUser(0,6)

leg = ROOT.TLegend(0.7, 0.75, 0.30, 0.87)
leg.AddEntry(r,"RVec","L")
leg.AddEntry(p,"OpenSYCL","L")
leg.AddEntry(g,"OneAPI","L")
leg.AddEntry(q,"CUDA","L")

c4.SetLogy();
c4.SetLogx();

r.Draw()
q.Draw('LP')
g.Draw('LP')
p.Draw('LP')
leg.Draw()
c4.Draw()

c4.SaveAs("Sexectimes.png")

Info in <TCanvas::Print>: png file Sexectimes.png has been created
